In [1]:
from collections import defaultdict
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
np.random.seed(1)
data = pd.read_csv('train.csv',header=0)
# print(list(data))
song_data = pd.read_csv('songs.csv',header=0)
print(list(song_data))
members= pd.read_csv('members.csv',header=0)
print(list(members))

['song_id', 'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist', 'language']
['msno', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date']


In [2]:
user_vals = defaultdict(int)
song_vals = defaultdict(int)
for i in range(len(data)):
    if i%100000 == 0:
        print(i)
    user_vals[data['msno'][i]] += 1
    song_vals[data['song_id'][i]] += 1
user_ = []
song_ = []
scount = []
ucount = []
for u in user_vals:
    user_.append(u)
    ucount.append(user_vals[u])
for s in song_vals:
    song_.append(s)
    scount.append(song_vals[s])

song_sorted = sorted(song_vals.items(), key=lambda x:x[1], reverse=True)
user_sorted = sorted(user_vals.items(), key=lambda x:x[1], reverse=True)

def get_top_vals(data,thres):
    if data == 'songs':
        base = song_sorted[:]
    elif data == 'users':
        base = user_sorted[:]
    ret = set(base[:thres])
    return ret

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000


In [3]:
#Generate Dictionaries for songs

genre_list = defaultdict(int)
song_genre_list = dict(defaultdict(int))
artist_list = defaultdict(int)
language_list = defaultdict(int)
composer_list = defaultdict(int)
song_artist_list = defaultdict(str)
song_language_list = defaultdict(str)
song_composer_list = defaultdict(str)

for i in range(len(song_data)):
    s = song_data['song_id'][i]

    if s not in song_genre_list:
        song_genre_list[s] = defaultdict(int)
    if  not pd.isnull(song_data.loc[i,'genre_ids']):
        genlist = song_data['genre_ids'][i].split('|')
        #print(genlist)
        for j in range(len(genlist)):
            song_genre_list[s][genlist[j]] = 1
            genre_list[genlist[j]] += 1

    song_artist_list[s] = song_data['artist_name'][i]
    artist_list[song_data['artist_name'][i]] += 1

    if not pd.isnull(song_data.loc[i,'language']) and song_data['language'][i] != -1:
        song_language_list[s] = song_data['language'][i]
        language_list[song_data['language'][i]] += 1

    if  not pd.isnull(song_data.loc[i,'composer']):
        song_composer_list[s] = song_data['composer'][i]
        composer_list[song_data['composer'][i]] += 1



In [4]:
#Generate Dictionaries for members

gender_list = defaultdict(int)
city_list = defaultdict(int)
bd_list = defaultdict(int)
user_gender_list = defaultdict(str)
user_city_list = defaultdict(int)
user_bd_list = defaultdict(int)

for i in range(len(members)):
    u = members['msno'][i]
    
    if not pd.isnull(members.loc[i,'gender']):
        user_gender_list[u] = members['gender'][i]
        gender_list[members['gender'][i]] += 1
    
    user_city_list[u] = members['city'][i]
    city_list[members['city'][i]] += 1
    
    if members['bd'][i] != 0:
        user_bd_list[u] = members['bd'][i]
        bd_list[members['bd'][i]] += 1


In [5]:
### Data Sampling
from random import randrange, uniform
random_songs=set()
while(len(random_songs)<1000000):
    x=randrange(0,len(song_data))
    random_songs.add(song_data['song_id'][x])

random_users=set()
while(len(random_users)<10000):
    x=randrange(0,len(members))
    u = members['msno'][x]
    if u in user_bd_list and u in user_gender_list:
        random_users.add(u)
print(len(random_users))
X_sample=[]
Y_sample=[]
for i in range(len(data)):
    if data['msno'][i] in random_users and data['song_id'][i] in random_songs:
        given_user=data['msno'][i]
        X_sample.append((data['msno'][i], data['song_id'][i]))
        Y_sample.append(data['target'][i])    
print(len(X_sample))

10000
1364683


In [6]:
# Random Samples
from random import randrange, uniform
#from sets import Set
X_samplep = []
X_samplen = []
Y_samplep = []
Y_samplen = []
N = 650000
for i in range(len(X_sample)):
    if Y_sample[i] == 0:
        X_samplen.append(X_sample[i])
        Y_samplen.append(Y_sample[i])
    else:
        X_samplep.append(X_sample[i])
        Y_samplep.append(Y_sample[i])
X = X_samplen[:int(N*0.6)] + X_samplep[:int(N*0.6)]
Y = Y_samplen[:int(N*0.6)] + Y_samplep[:int(N*0.6)]
X_valid = X_samplen[int(N*0.6):int(N*0.8)] + X_samplep[int(N*0.6):int(N*0.8)]
Y_valid = Y_samplen[int(N*0.6):int(N*0.8)] + Y_samplep[int(N*0.6):int(N*0.8)]
X_test = X_samplen[int(N*0.8):N] + X_samplep[int(N*0.8):N]
Y_test = Y_samplen[int(N*0.8):N] + Y_samplep[int(N*0.8):N]

In [7]:
# Data Stats
from collections import defaultdict
dd = defaultdict(int)
ss = defaultdict(int)
for i in range(len(X)):
    dd[X[i][0]]+=1
    ss[X[i][1]]+=1
avg = 0
for u in dd:
    avg += dd[u]
avg = avg * 1.0 / len(dd)
print('num_users: ', len(dd))
print('num_songs: ', len(ss))
print(avg)
print(sum(Y))
print(len(X))

num_users:  8461
num_songs:  73582
92.18768467084269
390000
780000


In [8]:
songs =[(song_data['song_id'][i],song_data['song_length'][i],song_data['genre_ids'][i],
     song_data['artist_name'][i],song_data['composer'][i],song_data['lyricist'][i],
    song_data['language'][i],) for i in range(len(song_data))]

In [9]:
## User User Similarity based on song

song_list = defaultdict(set)
user_list=dict(defaultdict(str))

for d in range(len(X)):
    user = X[d][0]
    song = X[d][1]
    if Y[d]==1:
        song_list[song].add(user)

for i in song_list:
    s=song_list[i]
    for u1 in s:
        for u2 in s:
            if u1!=u2:
                if u1 in user_list:
                    if u2 in user_list[u1]:
                        user_list[u1][u2] +=1
                    else:
                        user_list[u1][u2]=1
                else:
                    user_list[u1]=defaultdict(str)
                    user_list[u1][u2]=1
                    

In [10]:
## Song Song Similarity based on users
users_songlist=defaultdict(set)
songs_songs_list=dict(defaultdict(str))

for d in range(len(X)):
    user = X[d][0]
    song = X[d][1]
    if Y[d]==1:
        users_songlist[user].add(song)

for i in users_songlist:
    u=users_songlist[i]
    for s1 in u:
        for s2 in u:
            if s1!=s2:
                if s1 in songs_songs_list:
                    if s2 in songs_songs_list[s1]:
                        songs_songs_list[s1][s2] +=1
                    else:
                        songs_songs_list[s1][s2]=1
                else:
                    songs_songs_list[s1]=defaultdict(str)
                    songs_songs_list[s1][s2]=1




In [11]:
## User Genre Dict
user_genre_list = defaultdict(set)
for x in range(len(X)):
    u = X[x][0]
    s = X[x][1]
    if s in song_genre_list:
        for g in song_genre_list[s]:
            user_genre_list[u].add(g)

In [12]:
#Generate Threshhold Dictionaries
gender_genre_list = dict(defaultdict(int))
gender_artist_list = dict(defaultdict(int))
gender_composer_list = dict(defaultdict(int))
bd_genre_list = dict(defaultdict(int))
bd_artist_list = dict(defaultdict(int))
city_genre_list = dict(defaultdict(int))
city_language_list = dict(defaultdict(int))
city_artist_list = dict(defaultdict(int))

for i in range(len(X)):
    u = X[i][0]
    s = X[i][1]
    #print u,s
    if u in user_gender_list:
#         print('Hi')
        if user_gender_list[u] not in gender_genre_list:
#             print('Hi')
            gender_genre_list[user_gender_list[u]] = defaultdict(int)
        if s in song_genre_list:
            for j in song_genre_list[s]:
                gender_genre_list[user_gender_list[u]][j] += 1
        if user_gender_list[u] not in gender_composer_list:
            gender_composer_list[user_gender_list[u]] = defaultdict(int)
        if s in song_composer_list:
            gender_composer_list[user_gender_list[u]][song_composer_list[s]] += 1
        if user_gender_list[u] not in gender_artist_list:
            gender_artist_list[user_gender_list[u]] = defaultdict(int)
        gender_artist_list[user_gender_list[u]][song_artist_list[s]] += 1
    
    if u in user_bd_list:
        if user_bd_list[u] not in bd_genre_list:
            bd_genre_list[user_bd_list[u]] = defaultdict(int)
        if s in song_genre_list:
            for j in song_genre_list[s]:
                bd_genre_list[user_bd_list[u]][j] += 1
        if user_bd_list[u] not in bd_artist_list:
            bd_artist_list[user_bd_list[u]] = defaultdict(int)
        bd_artist_list[user_bd_list[u]][song_artist_list[s]] += 1
    
    if user_city_list[u] not in city_genre_list:
        city_genre_list[user_city_list[u]] = defaultdict(int)
    if s in song_genre_list:
        for j in song_genre_list[s]:
            city_genre_list[user_city_list[u]][j] += 1
    if user_city_list[u] not in city_language_list:
        city_language_list[user_city_list[u]] = defaultdict(int)
    city_language_list[user_city_list[u]][song_language_list[s]] += 1
    if user_city_list[u] not in city_artist_list:
        city_artist_list[user_city_list[u]] = defaultdict(int)
    city_artist_list[user_city_list[u]][song_artist_list[s]] += 1


In [13]:
#Generate User Artist similarity

artist_userlist=defaultdict(set)
sim_users_art=dict(defaultdict(str))

for d in range(len(X)):
    user = X[d][0]
    song = X[d][1]
    if Y[d]==1:
        artist= song_artist_list[song]
        artist_userlist[artist].add(user)

for i in artist_userlist:
    a=artist_userlist[i]
    for u1 in a:
        for u2 in a:
            if u1!=u2:
                if u1 in sim_users_art:
                    if u2 in sim_users_art[u1]:
                        sim_users_art[u1][u2] +=1
                    else:
                        sim_users_art[u1][u2]=1
                else:
                    sim_users_art[u1]=defaultdict(str)
                    sim_users_art[u1][u2]=1

In [14]:
# User Language Similarity
language_userlist=defaultdict(set)
sim_users_lang=dict(defaultdict(str))

for d in range(len(X)):
    user = X[d][0]
    song = X[d][1]
    lang= song_language_list[song]
    if Y[d]==1:
        language_userlist[lang].add(user)



# for i in language_userlist:
#     l=language_userlist[i]
#     for u1 in l:
#         for u2 in l:
#             if u1!=u2:
#                 if u1 in sim_users_lang:
#                     if u2 in sim_users_lang[u1]:
#                         sim_users_lang[u1][u2] +=1
#                     else:
#                         sim_users_lang[u1][u2]=1
#                 else:
#                     sim_users_lang[u1]=defaultdict(str)
#                     sim_users_lang[u1][u2]=1

In [15]:
#User Genre Similarity 

genre_userlist=defaultdict(set)
sim_users_genre=dict(defaultdict(str))

for d in range(len(X)):
    user = X[d][0]
    song = X[d][1]
    if Y[d]==1:
        if song in song_genre_list:
            genres= song_genre_list[song]
            for g in genres:
                    genre_userlist[g].add(user)
    

# for i in genre_userlist:
#     g=genre_userlist[i]
#     for u1 in g:
#         for u2 in g:
#             if u1!=u2:
#                 if u1 in sim_users_genre:
#                     if u2 in sim_users_genre[u1]:
#                         sim_users_genre[u1][u2] +=1
#                     else:
#                         sim_users_genre[u1][u2]=1
#                 else:
#                     sim_users_genre[u1]=defaultdict(str)
#                     sim_users_genre[u1][u2]=1    



            

# print(len(sim_users_genre))

In [16]:
gender_genre_sorted = defaultdict(list)
gender_artist_sorted = defaultdict(list)
gender_composer_sorted = defaultdict(list)
bd_genre_sorted = defaultdict(list)
bd_artist_sorted = defaultdict(list)
city_genre_sorted = defaultdict(list)
city_language_sorted = defaultdict(list)
city_artist_sorted = defaultdict(list)

for g in gender_genre_list:
    gender_genre_sorted[g] = sorted(gender_genre_list[g].items(), key=lambda x:x[1], reverse=True)
for g in gender_artist_list:
    gender_artist_sorted[g] = sorted(gender_artist_list[g].items(), key=lambda x:x[1], reverse=True)
for g in gender_composer_list:
    gender_composer_sorted[g] = sorted(gender_composer_list[g].items(), key=lambda x:x[1], reverse=True)
for b in bd_genre_list:
    bd_genre_sorted[b] = sorted(bd_genre_list[b].items(), key=lambda x:x[1], reverse=True)
for b in bd_artist_list:
    bd_artist_sorted[b] = sorted(bd_artist_list[b].items(), key=lambda x:x[1], reverse=True)
for c in city_genre_list:
    city_genre_sorted[c] = sorted(city_genre_list[c].items(), key=lambda x:x[1], reverse=True)
for c in city_language_list:
    city_language_sorted[c] = sorted(city_language_list[c].items(), key=lambda x:x[1], reverse=True)
for c in city_artist_list:
    city_artist_sorted[c] = sorted(city_artist_list[c].items(), key=lambda x:x[1], reverse=True)

In [45]:
bd_bucket = [0,15,30,45]
bd_bucket_genre_list = dict(defaultdict(int))
bd_bucket_artist_list = dict(defaultdict(int))
bd_genre_sorted = defaultdict(list)
bd_artist_sorted = defaultdict(list)
for b in bd_genre_list:
    if b <= bd_bucket[1]:
        buck = 0
    elif b <= bd_bucket[2]:
        buck = 1
    elif b <= bd_bucket[3]:
        buck = 2
    else:
        buck = 3
    if buck not in bd_bucket_genre_list:
        bd_bucket_genre_list[buck] = defaultdict(int)
    for i in bd_genre_list[b]:
        bd_bucket_genre_list[buck][i] += bd_genre_list[b][i]

for b in bd_artist_list:
    if b <= bd_bucket[1]:
        buck = 0
    elif b <= bd_bucket[2]:
        buck = 1
    elif b <= bd_bucket[3]:
        buck = 2
    else:
        buck = 3
    if buck not in bd_bucket_artist_list:
        bd_bucket_artist_list[buck] = defaultdict(int)
    for i in bd_artist_list[b]:
        bd_bucket_artist_list[buck][i] += bd_artist_list[b][i]

for b in bd_bucket_genre_list:
    bd_genre_sorted[b] = sorted(bd_bucket_genre_list[b].items(), key=lambda x:x[1], reverse=True)
for b in bd_bucket_artist_list:
    bd_artist_sorted[b] = sorted(bd_bucket_artist_list[b].items(), key=lambda x:x[1], reverse=True)

In [47]:
for b in bd_artist_sorted:
    for i in range(10):
        print(bd_artist_sorted[b][i])

('Various Artists', 119)
('BANGTAN BOYS', 96)
('林俊傑 (JJ Lin)', 80)
('周杰倫 (Jay Chou)', 71)
('玖壹壹', 54)
('Alan Walker', 47)
('五月天 (Mayday)', 46)
('TFBOYS', 44)
('Eric 周興哲', 42)
('TWICE', 41)
('Various Artists', 20911)
('周杰倫 (Jay Chou)', 16049)
('五月天 (Mayday)', 13575)
('林俊傑 (JJ Lin)', 8848)
('陳奕迅 (Eason Chan)', 8243)
('玖壹壹', 7159)
('aMEI (張惠妹)', 6417)
('田馥甄 (Hebe)', 6338)
('Alan Walker', 5899)
('Maroon 5', 5764)
('Various Artists', 7525)
('五月天 (Mayday)', 6090)
('周杰倫 (Jay Chou)', 5282)
('陳奕迅 (Eason Chan)', 3385)
('aMEI (張惠妹)', 3144)
('林俊傑 (JJ Lin)', 2733)
('玖壹壹', 2622)
('田馥甄 (Hebe)', 2353)
('Maroon 5', 1829)
('Alan Walker', 1809)
('Various Artists', 1915)
('五月天 (Mayday)', 1089)
('周杰倫 (Jay Chou)', 891)
('林俊傑 (JJ Lin)', 660)
('aMEI (張惠妹)', 502)
('陳奕迅 (Eason Chan)', 489)
('Maroon 5', 451)
('Alan Walker', 382)
('江蕙 (Jody Jiang)', 367)
('田馥甄 (Hebe)', 363)


In [17]:
def similarity_check(A,B,x,y):
    flag=False
    if x in A:
        for u in A[x]:
            if u in B[y]:
                return 1
    if flag==False:
            return 0 
        
    return 0

def threshold_check(dictionary,uval,sval,thres):
#     if x not in A or y not in B:
#         return 0
#     uval = A[x]
#     sval = B[y]
    ctr = 0
    for i in range(len(dictionary[uval])):
        ctr += 1
        if sval == dictionary[uval][i][0]:
            return 1
        if ctr > thres*len(dictionary[uval]):
            break
    return 0

In [18]:
from sklearn.metrics import silhouette_samples, silhouette_score
gl_count = 0
city_dict=dict(zip(city_list, range(len(city_list))))
bd_dict=dict(zip(bd_list, range(len(bd_list))))
gender_dict=dict(zip(gender_list, range(len(gender_list))))
genre_dict=dict(zip(genre_list, range(len(genre_list))))
language_dict= dict(zip(language_list, range(len(language_list))))

# user_songlist
users_list = [u for u in random_users]
from sklearn.cluster import KMeans
def kmeansfeatures(user):
    global gl_count
    gl_count+=1
    if gl_count % 100000 == 0:
        print(gl_count)
    feat1=[0]*(len(genre_list)+len(language_list))
    for song in users_songlist[user]:
        if song in song_genre_list:
            for i in song_genre_list[song]:
                curr_genre=genre_dict[i]
                feat1[curr_genre]+=1
        if song in song_language_list:
            if song_language_list[song] in language_dict:
                curr_lang=language_dict[song_language_list[song]]
                feat1[len(genre_list)+curr_lang]+=1
    return feat1

k_means = KMeans(n_clusters=30)
X_km = [kmeansfeatures(u) for u in users_list]
k_means_labels = k_means.fit_predict(X_km)
print(k_means.inertia_)

# silhouette_avg = silhouette_score(X_km, k_means_labels)
# print(silhouette_avg)

2974213.94533


In [19]:
user_cluster = {}
cluster_user = defaultdict(set)
for i in range(len(users_list)):
    u = users_list[i]
    cl = k_means_labels[i]
    user_cluster[u] = cl
    cluster_user[cl].add(u)
print( len(users_list))
print(len(k_means_labels))
for cl in cluster_user:
    print(cl, len(cluster_user[cl]))

10000
10000
0 3953
1 149
2 128
3 407
4 8
5 68
6 62
7 1292
8 488
9 177
10 351
11 55
12 110
13 302
14 148
15 228
16 27
17 11
18 815
19 9
20 203
21 36
22 71
23 5
24 528
25 1
26 29
27 262
28 64
29 13


In [51]:
#Extract Features
gl_count = 0
city_dict=dict(zip(city_list, range(len(city_list))))
bd_dict=dict(zip(bd_list, range(len(bd_list))))
gender_dict=dict(zip(gender_list, range(len(gender_list))))
genre_dict=dict(zip(genre_list, range(len(genre_list))))
language_dict= dict(zip(language_list, range(len(language_list))))


def feature1(datum):
    global gl_count
    gl_count+=1
#     bd_bucket = [0,15,30,45]
    if gl_count % 100000 == 0:
        print(gl_count)
    feat1=[0]*(len(city_list)+len(bd_bucket)+len(gender_list)+len(genre_list)+len(language_list))
#     feat1=[0]*(len(genre_list)+len(language_list))
#     feat1 = []
    ##print(feat1)
    user=datum[0]
    song=datum[1]
    if user in user_bd_list:
        pbdval = 1
    else:
        pbdval = 0
    if pbdval == 1:
        bdval = user_bd_list[user]
        if bdval <= bd_bucket[1]:
            buck = 0
        elif bdval <= bd_bucket[2]:
            buck = 1
        elif bdval <= bd_bucket[3]:
            buck = 2
        else:
            buck = 3
    
    curr_city=city_dict[user_city_list[user]]
    feat1[curr_city]=1
    if user in user_bd_list:
        curr_bd=bd_dict[user_bd_list[user]]
        flag = False
        for j in range(len(bd_bucket)-1):
            if curr_bd <= bd_bucket[j+1]:
                flag = True
                feat1[len(city_list)+j] = 1
                break
        if flag == False:
            feat1[len(city_list)+len(bd_bucket)-1] = 1
    if user in user_gender_list:
        curr_gender=gender_dict[user_gender_list[user]]
        feat1[len(city_list)+len(bd_bucket)+curr_gender]=1
    if song in song_genre_list:
        for i in song_genre_list[song]:
            curr_genre=genre_dict[i]
            feat1[len(city_list)+len(bd_bucket)+len(gender_list)+curr_genre]=1
    if song in song_language_list:
        if song_language_list[song] in language_dict:
            curr_lang=language_dict[song_language_list[song]]
            feat1[len(city_list)+len(bd_bucket)+len(gender_list)+len(genre_list)+curr_lang]=1
        
#      Append for user-user similarity based on song
    val=similarity_check(user_list,song_list,user,song)
#     print(val)
#     print("user-user sim",val)
    feat1.append(val)
    
    
    ##Append for song-song similarity based on user
    val=similarity_check(songs_songs_list,users_songlist,song,user)
#     print("song-song sim",val)
    feat1.append(val)
    
    ##Append for user user similarity based on Artist
#     if song in song_artist_list:
#         artist=song_artist_list[song]
#         val=similarity_check(sim_users_art,artist_userlist,user,artist)
# #         print("user-art sim",val)
#         feat1.append(val)
#     else:
#         feat1.append(0)
    
#     ##Append for user user similarity based on Genere
    if song in song_genre_list:
        flag=False
        for g in song_genre_list[song]:
            val = user in genre_userlist[g]  #similarity_check(sim_users_genre,genre_userlist,user,g)
            if val==1:
                feat1.append(1)
                flag=True
                break
        if flag==False:
            feat1.append(0)
    else:
        feat1.append(0)
    
#     ##Append for user-user similarity based on Language
    if song in song_language_list:
        lang=song_language_list[song]
        val = int(user in language_userlist[lang]) #similarity_check(sim_users_lang,language_userlist,user,lang)
        feat1.append(val)    
    else:
        feat1.append(0)
        
    ## Append Gender-Genre Threshold Value
    if user in user_gender_list and song in song_genre_list:
        flag = False
        for g in song_genre_list[song]:
            val = threshold_check(gender_genre_sorted,user_gender_list[user],g,0.1)
            if val == 1:
                feat1.append(1)
                flag = True
                break
        if flag == False:
            feat1.append(0)
    else:
        feat1.append(0)
    
    ## Append Gender-Artist Threshold
    if user in user_gender_list and song in song_artist_list:
        val = threshold_check(gender_artist_sorted,user_gender_list[user],song_artist_list[song],0.1)
        feat1.append(val)
    else:
        feat1.append(0)
     
    ## Append BD-Genre Threshold Value
    if pbdval == 1 and song in song_genre_list:
        flag = False
        for g in song_genre_list[song]:
            val = threshold_check(bd_genre_sorted,buck,g,0.1)
            if val == 1:
                feat1.append(1)
                flag = True
                break
        if flag == False:
            feat1.append(0)
    else:
        feat1.append(0)
    
    ## Append BD-Artist Threshold
    if pbdval == 1 and song in song_artist_list:
        val = threshold_check(bd_artist_sorted,buck,song_artist_list[song],0.1)
        feat1.append(val)
    else:
        feat1.append(0)
    
#     ## Append City-Genre Threshold Value
    if user in user_city_list and song in song_genre_list:
        flag = False
        for g in song_genre_list[song]:
            val = threshold_check(city_genre_sorted,user_city_list[user],g,0.1)
            if val == 1:
                feat1.append(1)
                flag = True
                break
        if flag == False:
            feat1.append(0)
    else:
        feat1.append(0)
    
#     ## Append City-Language Threshold
    if user in user_city_list and song in song_language_list:
        val = threshold_check(city_language_sorted,user_city_list[user],song_language_list[song],0.1)
        feat1.append(val)
    else:
        feat1.append(0)
    
#     ## Append City-Artist Threshold
    if user in user_city_list and song in song_artist_list:
        val = threshold_check(city_artist_sorted,user_city_list[user],song_artist_list[song],0.1)
        feat1.append(val)
    else:
        feat1.append(0)
    ## K-Means Genre Similarity
    if user not in user_cluster:
        feat1.append(0)
    else:
        c = user_cluster[user]
        if song in song_genre_list:
            flag = False
            for u in cluster_user[c]:
                for g in song_genre_list[song]:
                    if u in genre_userlist[g]:
                        flag = True
                        feat1.append(1)
                        break
                if flag == True:
                    break
            if flag == False:
                feat1.append(0)
        else:
            feat1.append(0)
    
    ## K-Means Language Similarity
    if user not in user_cluster:
        feat1.append(0)
    else:
        c = user_cluster[user]
        if song in song_language_list:
            flag = False
            if song_language_list[song] in language_dict:
                for u in cluster_user[c]:
                    if u in language_userlist[song_language_list[song]]:
                        flag = True
                        feat1.append(1)
                        break
            if flag==False:
                feat1.append(0)
        else:
            feat1.append(0)
    
    feat1.append(1)
    return feat1
    
    
    
# genre_userlist=defaultdict(set)
# sim_users_genre=dict(defaultdict(str))   



In [52]:
print(len(X))
gl_count = 0
X_train_feat=[feature1(d) for d in X]
X_train_feat = np.array(X_train_feat)
y_train = np.array(Y)
y_train = np.resize(y_train,(len(Y),1))
gl_count = 0
X_valid_feat=[feature1(d) for d in X_valid]
X_valid_feat = np.array(X_valid_feat)
y_valid = np.array(Y_valid)
y_valid = np.resize(y_valid,(len(Y_valid),1))
gl_count = 0
X_test_feat=[feature1(d) for d in X_test]
X_test_feat = np.array(X_test_feat)
y_test = np.array(Y_test)
y_test = np.resize(y_test,(len(Y_test),1))

780000
100000
200000
300000
400000
500000
600000
700000
100000
200000
100000
200000


In [ ]:
logreg = LogisticRegression(tol=0.0000001,solver='sag',max_iter=30000, C=100000)
logreg.fit(X_train_feat, y_train)


pred = logreg.predict(X_train_feat)
res = []
for i in range(pred.shape[0]):
    res.append(pred[i] == y_train[i])
acc = 1.0*sum(res)/len(res)
print('training accuracy ', acc)


pred = logreg.predict(X_valid_feat)
res = []
for i in range(pred.shape[0]):
    res.append(pred[i] == y_valid[i])
acc = 1.0*sum(res)/len(res)
print('Validation accuracy: ', acc)

/home/roshan/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from sklearn.metrics import roc_auc_score
pred = logreg.predict(X_test_feat)
auc_test = roc_auc_score(Y_test,pred)
print(auc_test)

In [ ]:
print(len(X))

In [50]:
X_train_feat[:5]

array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]])